In [1]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, classification_report, f1_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')

data = pd.read_csv('Data/25day_dataset.csv')


C:\Users\danie\AppData\Local\Temp\ipykernel_6516\1430184262.py:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


Si etichetta il dataset con 4 classi:

    1) Tutti spenti [0]
    2) Solo lavatrice accesa [1]
    3) Solo lavastoviglie accesa [2]
    4) Solo forno acceso [3]

In [2]:
label = []
for i in range(len(data)):
    if data['wahing_machine'].iloc[i]==0 and data['dishwasher'].iloc[i]==0 and data['oven'].iloc[i]==0:
        label.append(0)
    elif data['wahing_machine'].iloc[i]>0:
        label.append(1)
    elif data['dishwasher'].iloc[i]>0:
        label.append(2)
    elif data['oven'].iloc[i]>0:
        label.append(3)
data['Class'] = label

In [3]:
#Droppa le colonne che non servono piu 
data.drop(['wahing_machine', 'dishwasher', 'oven'], axis=1, inplace=True)

di seguito le istruzioni fronite per effettuare il corretto folding:

Test set fold 01: 
Tutte le misure del giorno 2022-01-01


Test set fold 02: 
Tutte le misure del giorno 2022-01-02

Test set fold 03: 
Tutte le misure del giorno 2022-01-03

…..


Test set fold 09: 
Tutte le misure del giorno 2022-01-10
Tutte le misure del giorno 2022-01-11

Test set fold 10: 
Tutte le misure del giorno 2022-01-12
Tutte le misure del giorno 2022-01-13 (dati disponibili fino alle 03:16)


In [4]:
test_set_list=[]
Test_set1 =data[data['DateTime'].str.startswith('2022-01-01')]
Test_set2 =data[data['DateTime'].str.startswith('2022-01-02')]
Test_set3 =data[data['DateTime'].str.startswith('2022-01-03')]
Test_set4 =data[data['DateTime'].str.startswith('2022-01-04')]
Test_set5 =data[data['DateTime'].str.startswith('2022-01-05')]
Test_set6 =data[data['DateTime'].str.startswith('2022-01-06')]
Test_set7 =data[data['DateTime'].str.startswith('2022-01-07')]
Test_set8 =data[(data['DateTime'].str.startswith('2022-01-08')) | (data['DateTime'].str.startswith('2022-01-09'))]
Test_set9 =data[(data['DateTime'].str.startswith('2022-01-10')) | (data['DateTime'].str.startswith('2022-01-11'))]
Test_set10 =data[(data['DateTime'].str.startswith('2022-01-12')) | (data['DateTime'].str.startswith('2022-01-13'))]

In [7]:
test_set_list= [Test_set1, Test_set2, Test_set3, Test_set4, Test_set5, Test_set6, Test_set7, Test_set8, Test_set9, Test_set10]
Test_set=Test_set1.append([Test_set2,Test_set3,Test_set4,Test_set5,Test_set6,Test_set7,Test_set8,Test_set9,Test_set10])
Training_set= data.drop(Test_set.index, axis=0)

C:\Users\danie\AppData\Local\Temp\ipykernel_6516\1407526483.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Test_set=Test_set1.append([Test_set2,Test_set3,Test_set4,Test_set5,Test_set6,Test_set7,Test_set8,Test_set9,Test_set10])


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(Training_set.drop(['DateTime', 'Class'], axis=1), Training_set.Class, test_size=0.025, random_state= 8) 

Validation per settare i parametri ottimali della random forest

In [20]:
# validation
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rfc=RandomForestClassifier(random_state=42)
param_grid = { 
    'n_estimators': [100, 300, 500],
    'min_samples_leaf': [50, 100, 150],
    'criterion' : ['gini', 'entropy', 'log_loss']
}
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_val, y_val)
CV_rfc.best_params_

{'criterion': 'entropy', 'min_samples_leaf': 50, 'n_estimators': 100}

Undersampling

In [22]:
from imblearn.under_sampling  import RandomUnderSampler
from collections import Counter
print(Counter(y_train), y_train.count())
n= 400000 - (y_train.values == 1).sum() - (y_train.values == 3).sum() - (y_train.values == 2).sum()
undersample = RandomUnderSampler(sampling_strategy={0: n, 1: (y_train.values == 1).sum(), 2: (y_train.values == 2).sum(), 3: (y_train.values == 3).sum()})
X_new, y_new = undersample.fit_resample(X_train, y_train)
print(Counter(y_new), y_new.count())

Counter({0: 952378, 1: 41140, 2: 13344, 3: 4018}) 1010880
Counter({0: 341498, 1: 41140, 2: 13344, 3: 4018}) 400000


Training

In [26]:
#random froset:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, criterion='entropy', min_samples_leaf=50)
clf.fit(X_new, y_new)
y_preds= clf.predict(Test_set.drop(['DateTime', 'Class'], axis=1))

print("CM:\n" + str(confusion_matrix(Test_set.Class,y_preds)) + "\n")
print(classification_report(Test_set.Class,y_preds))

CM:
[[1055382    2402      92      91]
 [   4294   31437       1       0]
 [   4378     110   10162       1]
 [   3533     798       2   10517]]

              precision    recall  f1-score   support

           0       0.99      1.00      0.99   1057967
           1       0.90      0.88      0.89     35732
           2       0.99      0.69      0.82     14651
           3       0.99      0.71      0.83     14850

    accuracy                           0.99   1123200
   macro avg       0.97      0.82      0.88   1123200
weighted avg       0.99      0.99      0.99   1123200



In [17]:
# ensamble di alberi decisionali one vs one
from sklearn.multiclass import OneVsOneClassifier
from sklearn import tree

Training_set= data.drop(Test_set.index, axis=0)

model = tree.DecisionTreeClassifier()
ovo = OneVsOneClassifier(model)
ovo.fit(X_new, y_new)
predicted = ovo.predict(Test_set.drop(['DateTime', 'Class'], axis=1))

print("CM:\n" + str(confusion_matrix(Test_set.Class,predicted)) + "\n")
print(classification_report(Test_set.Class,predicted))

CM:
[[66047 13568  1449    31]
 [  130   892     0     2]
 [  843   131  2603     0]
 [    1   641     1    61]]

              precision    recall  f1-score   support

           0       0.99      0.81      0.89     81095
           1       0.06      0.87      0.11      1024
           2       0.64      0.73      0.68      3577
           3       0.65      0.09      0.15       704

    accuracy                           0.81     86400
   macro avg       0.58      0.62      0.46     86400
weighted avg       0.96      0.81      0.87     86400

